# Librerías

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from google.colab import drive
import os

# Cargamos la base de datos previamente procesada en el anterior notebook

In [ ]:

drive.mount('/content/drive', force_remount=True)
os.chdir("/content/drive/MyDrive/ProyectoModelos1/data")

Mounted at /content/drive


In [ ]:
df= pd.read_csv('clean_train.csv')

## Para este entrenamiento, solo podemos utilizar columnas con valores numericos o booleanos. Para esto nos deshacemos de las columnas que no cumplen estos requisitos

In [ ]:
df = df.drop(columns=['PERIODO', 'ESTU_PRGM_ACADEMICO', 'FAMI_ESTRATOVIVIENDA',
       'FAMI_EDUCACIONPADRE', 'FAMI_EDUCACIONMADRE'])

### Luego mapeamos los valores de la columna RENDIMIENTO_GLOBAL con valores del 0 al 3, esto necesario para poder realizar el entrenamiento porque, como ya se mencionó anteriormente, el modelo solo acepta valores numericos o booleanos.

In [ ]:
# Crear un diccionario de mapeo
mapeo = {
    'bajo': 0,
    'medio-bajo': 1,
    'medio-alto': 2,
    'alto': 3
}

# Aplicar el mapeo a la columna
df['RENDIMIENTO_GLOBAL'] = df['RENDIMIENTO_GLOBAL'].map(mapeo)

### Procedemos entonces a realizar el entrenamiento.

define un modelo RandomForestClassifier con 80 estimadores (árboles), utiliza el criterio de Gini para medir la calidad de las divisiones, requiere al menos 8 muestras para dividir un nodo interno, considera la raíz cuadrada del número de características en cada división y establece una semilla aleatoria de 20 para reproducibilidad.

Luego definimos los dataFrame para proceder con el entrenamiento del modelo, el X_train con todas las caracteristicas exceptuando la columna a predecir y el y_train que contiene solo la columna a predecir. inicializamos el modelo con el impulso de ronda que ya definimos y los parametros para proceder a entrenarlo mandando X_train y y_train como parámetros.

In [ ]:
X_train = df.drop(columns=['RENDIMIENTO_GLOBAL'])
y_train = df['RENDIMIENTO_GLOBAL']

clf = RandomForestClassifier(n_estimators=80,
                             criterion='gini',
                             min_samples_split=8,
                             max_features='sqrt',
                             random_state=20)
clf.fit(X_train, y_train)



RandomForestClassifier(min_samples_split=8, n_estimators=80, random_state=20)

## Ahora cargamos el dataFrame para testear la predicción del modelo. Procedemos tambien a eliminar las mismas columnas que borramos en el dataFrame de entrenamiento.

In [ ]:
new_df = pd.read_csv('clean_test.csv')

In [ ]:
new_df = new_df.drop(columns=['PERIODO', 'ESTU_PRGM_ACADEMICO', 'FAMI_ESTRATOVIVIENDA',
       'FAMI_EDUCACIONPADRE', 'FAMI_EDUCACIONMADRE'])

### Realizamos la predicción enviando como parametro el dataFrame de test y la almacenamos en una variable.

In [ ]:
predictions = clf.predict(new_df)

#### Le añadimos una nueva columna al dataFrame de test donde estan contenidos los valores que predecimos. Hacemos el mapeo inverso para obtener de los numeros los valores en String de las predicciones.

In [ ]:
new_df['RENDIMIENTO_GLOBAL'] = predictions

In [ ]:
mapeo_original = {
    0: 'bajo',
    1: 'medio-bajo',
    2: 'medio-alto',
    3: 'alto'
}

new_df['RENDIMIENTO_GLOBAL'] = new_df['RENDIMIENTO_GLOBAL'].map(mapeo_original)


### Procedemos a almacenar el id junto a la predicción en un nuevo dataFrame para descargarlo y subirlo a Kaggle.

In [ ]:
df_kaggle = new_df.loc[:, ['ID', 'RENDIMIENTO_GLOBAL']]
df_kaggle

,ID,RENDIMIENTO_GLOBAL
0,550236,medio-alto
1,98545,medio-alto
2,499179,medio-alto
3,782980,medio-bajo
4,785185,medio-bajo
...,...,...
296781,496981,medio-alto
296782,209415,medio-alto
296783,239074,alto
296784,963852,medio-alto


In [ ]:
df_kaggle = df_kaggle.rename(columns={'PREDICCION':'RENDIMIENTO_GLOBAL'})

In [ ]:
df_kaggle.to_csv('prediction_with_RandomForest.csv',index=False)
from google.colab import files
files.download('prediction_with_RandomForest.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>